# Experiment to perform GCA on LG for LP

In [3]:
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling, dropout_edge
from torch_geometric.transforms import LineGraph
import torch
import numpy as np
import random
import nni

from pGRACE.model import Encoder, GRACE
from pGRACE.functional import drop_feature, drop_edge_weighted, \
    degree_drop_weights, \
    evc_drop_weights, pr_drop_weights, \
    feature_drop_weights, drop_feature_weighted_2, feature_drop_weights_dense
from pGRACE.eval import log_regression, MulticlassEvaluator
from pGRACE.utils import get_base_model, get_activation, \
    generate_split, compute_pr, eigenvector_centrality
from pGRACE.dataset import get_dataset
from torch_geometric.datasets import AttributedGraphDataset
from torch_geometric.utils import degree, to_undirected
from pGRACE.functional import degree_drop_weights, feature_drop_weights

In [4]:
attr_graph_dataset = AttributedGraphDataset("./dataset", "Cora")
cora = attr_graph_dataset[0]
print(cora)
G = cora
dataset = attr_graph_dataset

Data(x=[2708, 1433], edge_index=[2, 5429], y=[2708])


In [ ]:
def all_negative_edges(edge_index, num_nodes):
    adj_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.bool)
    adj_matrix[edge_index[0], edge_index[1]] = True
    neg_edge_index = torch.nonzero(~adj_matrix, as_tuple=False).t()

    return neg_edge_index

In [ ]:
def to_linegraph(graph: Data):
    graph_c = graph.clone()
    num_edges = graph_c.num_edges
    # on construit le line graph_c avec 50% de negative edge et 50% de positive edge
    # on essaie de garder le nombre total de edge identique
    neg_edge_index = all_negative_edges(graph_c.ed
    graph_c.edge_index = torch.cat((pos_edge_index, neg_edge_index), -1)
    num_edges = graph_c.num_edges
    src_nodes = graph_c.edge_index[0]
    target_nodes = graph_c.edge_index[1]
    edge_attr = (
        graph_c.x[src_nodes] + graph_c.x[target_nodes]
    )  # aggregation des attributs de noeuds
    graph_c.edge_attr = edge_attr
    linegraph = LineGraph()(graph_c)
    edge_index_ = to_undirected(linegraph.edge_index)
    node_deg = degree(edge_index_[1])
    # print(node_deg.shape[0], linegraph.x.shape[0])
    if node_deg.shape[0] != linegraph.x.shape[0]:
        # print('aie')
        return to_linegraph(graph)
    linegraph.y = torch.tensor(
        np.concatenate((np.ones(pos_edge_index.size(1)), np.zeros(neg_edge_index.size(1)))), dtype=torch.int64
    )  # label la moitier son vrai, l'autre neg
    # print(linegraph)
    return linegraph

In [ ]:
LG = to_linegraph(G)
LG

In [12]:
param = {
    "learning_rate": 0.01,
    "num_hidden": 256,
    "num_proj_hidden": 32,
    "activation": "prelu",
    "base_model": "GCNConv",
    "num_layers": 2,
    "drop_edge_rate_1": 0.3,
    "drop_edge_rate_2": 0.4,
    "drop_feature_rate_1": 0.1,
    "drop_feature_rate_2": 0.0,
    "tau": 0.4,
    "num_epochs": 3000,
    "weight_decay": 1e-5,
    "drop_scheme": "degree",
}
device = 'cuda:0'
torch.manual_seed(12345)
device = torch.device(device)
random.seed(12345)
data = LG.to(device)

split = generate_split(
    data.num_nodes, train_ratio=0.1, val_ratio=0.1
)  # generic train test split

encoder = Encoder(
    len(data.x[1]),
    param["num_hidden"],
    get_activation(param["activation"]),
    base_model=get_base_model(param["base_model"]),
    k=param["num_layers"],
).to(device)

model = GRACE(
    encoder, param["num_hidden"], param["num_proj_hidden"], param["tau"]
).to(device)  # init the model

optimizer = torch.optim.Adam(
    model.parameters(), lr=param["learning_rate"], weight_decay=param["weight_decay"]
)

drop_weights = degree_drop_weights(data.edge_index).to(device)
edge_index_ = to_undirected(data.edge_index)
node_deg = degree(edge_index_[1])
feature_weights = feature_drop_weights(data.x, node_c=node_deg).to(device)

In [13]:
def train():
    model.train()
    optimizer.zero_grad()

    def drop_edge(idx: int):
        
        if param['drop_scheme'] == 'uniform':
            return dropout_adj(data.edge_index, p=param[f'drop_edge_rate_{idx}'])[0]
        elif param['drop_scheme'] in ['degree', 'evc', 'pr']:
            return drop_edge_weighted(data.edge_index, drop_weights, p=param[f'drop_edge_rate_{idx}'], threshold=0.7)
        else:
            raise Exception(f'undefined drop scheme: {param["drop_scheme"]}')

    edge_index_1 = drop_edge(1)
    edge_index_2 = drop_edge(2)
    x_1 = drop_feature(data.x, param['drop_feature_rate_1'])
    x_2 = drop_feature(data.x, param['drop_feature_rate_2'])

    if param['drop_scheme'] in ['pr', 'degree', 'evc']:
        x_1 = drop_feature_weighted_2(data.x, feature_weights, param['drop_feature_rate_1'])
        x_2 = drop_feature_weighted_2(data.x, feature_weights, param['drop_feature_rate_2'])

    z1 = model(x_1, edge_index_1)
    z2 = model(x_2, edge_index_2)

    loss = model.loss(z1, z2)
    loss.backward()
    optimizer.step()

    return loss.item()

In [14]:
def test(final=False):
    model.eval()
    
    z = model(data.x, data.edge_index)
    z = z.to(device)

    evaluator = MulticlassEvaluator()
    acc = log_regression(z, data, evaluator, split='rand:0.1', num_epochs=3000, preload_split=split)['acc']

    if final:
        nni.report_final_result(acc)
    else:
        nni.report_intermediate_result(acc)

    return acc

In [15]:
for epoch in range(1, param["num_epochs"] + 1):
    loss = train()
    if epoch % 10 == 0:
        print(f"(T) | Epoch={epoch:03d}, loss={loss:.4f}")

    if epoch % 100 == 0:
        acc = test()
        print(f"(E) | Epoch={epoch:04d}, avg_acc = {acc}")

(T) | Epoch=010, loss=9.2902
(T) | Epoch=020, loss=9.1153
(T) | Epoch=030, loss=9.0941
(T) | Epoch=040, loss=8.9735
(T) | Epoch=050, loss=8.9161
(T) | Epoch=060, loss=8.8862
(T) | Epoch=070, loss=8.7450
(T) | Epoch=080, loss=8.6569
(T) | Epoch=090, loss=8.7053
(T) | Epoch=100, loss=8.6950
[2025-02-21 13:37:54] Intermediate result: 0.593317985534668  (Index 4)
(E) | Epoch=0100, avg_acc = 0.593317985534668
(T) | Epoch=110, loss=8.6775
(T) | Epoch=120, loss=8.5511
(T) | Epoch=130, loss=8.4888
(T) | Epoch=140, loss=8.4160
(T) | Epoch=150, loss=8.3984
(T) | Epoch=160, loss=8.3025
(T) | Epoch=170, loss=8.3841
(T) | Epoch=180, loss=8.2801
(T) | Epoch=190, loss=8.2963
(T) | Epoch=200, loss=8.2268
[2025-02-21 13:38:01] Intermediate result: 0.6172811388969421  (Index 5)
(E) | Epoch=0200, avg_acc = 0.6172811388969421
(T) | Epoch=210, loss=8.1852
(T) | Epoch=220, loss=8.1453
(T) | Epoch=230, loss=8.1763
(T) | Epoch=240, loss=8.1283
(T) | Epoch=250, loss=8.1400
(T) | Epoch=260, loss=8.0421
(T) | Ep

KeyboardInterrupt: 

In [ ]:
acc = test()
print(f"{acc}")